En este cuaderno vamos a generar una interfaz de usuario para que las expertas puedan evaluar, validar y corregir los pares p-r generados por los modelos.

In [22]:
import pandas as pd

d = {'Pregunta': ["Hola", "Adios", "Eliminar", "Hasta luego"], 'Respuesta': ["Pablo", "Luis", "Eliminar", "Pedro"]}

df = pd.DataFrame(data=d)
df.to_csv('datos/corpus/Pablo.csv') 

In [46]:
d = {'Index': [0,1,2,3], 'Pregunta': ["Hola", "Adios", "Eliminar", "Hasta luego"], 'Respuesta': ["Pablo", "Luis", "Eliminar", "Pedro"]}
d2 = {'Index': [0],'Pregunta': ["Holi"], 'Respuesta': ["Pablito"]}

df = pd.DataFrame(data=d)
df2 = pd.DataFrame(data=d2)

#df.merge(df2, on="Index", how="left").ffill(axis=1).drop("Pregunta", axis=1)
df.update(df2)

In [51]:
aux = df.loc[1:2]
aux.loc[1,['Pregunta']] = "Adiosito"

In [52]:
aux

,Index,Pregunta,Respuesta
1,1.0,Adiosito,Luis
2,2.0,Eliminar,Eliminar


In [65]:
df.update(aux)

In [66]:
df

,Index,Pregunta,Respuesta
0,0.0,Holi,Pablito
1,1.0,Adiosito,Luis
2,2.0,Eliminar,Eliminar
3,3.0,Hasta luego,Pedro


In [65]:
import numpy as np
np.version.version

'1.24.4'

In [ ]:
from flask import *
from flask_socketio import SocketIO, emit
import pandas as pd
  
# creates a Flask application 
app = Flask(__name__, static_folder="static")
app.secret_key = "asdf1234"
socketio = SocketIO(app, async_mode='threading')
global df
global aux

def guardar_cambios(df, index, ruta, ident):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.to_csv(f'{ruta}/corpus/{ident}.csv')
    with open(f"{ruta}/checkpoint/{ident}.txt", "w") as output:
            output.write(str(index))

def actualizar_cambios(df, pregunta, respuesta, index):
    if pregunta and respuesta:
        df.loc[index,['Pregunta']] = pregunta
        df.loc[index,['Respuesta']] = respuesta
            
@app.route("/", methods=['GET', 'POST']) 
def index():
    global df
    global aux
    if 'ident' in session:
        ident = session['ident']
        if 'index' not in session:
            with open(f'datos/checkpoint/{ident}.txt') as f:
                index = f.readline()
                session["index"] = index
        if 'df' not in session:
            df = pd.read_csv(f'datos/corpus/{ident}.csv')
            aux = None
            session['df'] = True
        else:
            index = session["index"]
        guardar_cambios(df, index, "datos", ident)
        row = df.iloc[[index]]
        indice1 = "Registro "
        indice2 = f"de {df.shape[0]}"
        return render_template('index.html', t=ident, pregunta=row["Pregunta"].item(), 
                               respuesta=row["Respuesta"].item(), indice1=indice1, index=int(index)+1, 
                               indice2=indice2) 
    else:
        return render_template('index.html', t="", pregunta="", respuesta="", 
                               indice1="", index="", indice2="")

# Para incrementar el índice y pasar al siguiente registro
@app.route('/siguiente')
def siguiente():
    global df
    global aux
    if 'index' in session:
        index = int(session["index"])
        try:
            mensaje = json.loads(request.args['messages'])
            pregunta = mensaje['Pregunta']
            respuesta = mensaje['Respuesta']
            actualizar_cambios(df, pregunta, respuesta, int(index))
        except:
            pass
        if df.shape[0]-1 > index:
            index += 1
            session["index"] = index
        if aux is not None:
            aux = None
        return redirect(url_for('index'))
    else:
        return redirect(url_for('index'))

# Para decrementar el índice y pasar al anterior registro
@app.route('/anterior')
def anterior():
    global df
    global aux
    if 'index' in session:
        index = int(session["index"])
        try:
            mensaje = json.loads(request.args['messages'])
            pregunta = mensaje['Pregunta']
            respuesta = mensaje['Respuesta']
            actualizar_cambios(df, pregunta, respuesta, int(index))
        except:
            pass
        if 0 < index:
            index -= 1
            session["index"] = index
        if aux is not None:
            aux = None
        return redirect(url_for('index'))
    else:
        return redirect(url_for('index'))
    
# Para navegar cuando se ha cambiado el índice
@app.route('/navegar', methods=['GET', 'POST'])
def navegar():
    global df
    global aux
    if 'index' in session:
        nuevo = request.form['indice']
        try:
            nuevo = int(nuevo)
            nuevo = nuevo-1
        except:
            nuevo = session["index"]
        if 0 <= nuevo and nuevo < df.shape[0]:
            index = nuevo
            session["index"] = index
        if aux is not None:
            aux = None
        return redirect(url_for('index'))
    else:
        return redirect(url_for('index'))

@app.route('/actualizar', methods=['GET', 'POST'])
def actualizar():
    global df
    global aux
    if 'index' in session:
        index = int(session["index"])
        try:
            mensaje = json.loads(request.args['messages'])
            pregunta = mensaje['Pregunta']
            respuesta = mensaje['Respuesta']
            actualizar_cambios(df, pregunta, respuesta, int(index))
        except:
            pass
        if aux is not None:
            aux = None
    return redirect(url_for('index'))

# Para actualizar el contenido del dataframe con los nuevos valores de los campos de texto
@app.route('/generica', methods=['GET', 'POST'])
def generica():
    pregunta = request.form.get('pregunta')
    respuesta = request.form.get('respuesta')
    mensaje = json.dumps({"Pregunta": pregunta, "Respuesta": respuesta})
    if request.form['submit'] == 'Siguiente':
        return redirect(url_for('siguiente', messages=mensaje))
    elif request.form['submit'] == 'Anterior':
        return redirect(url_for('anterior', messages=mensaje))
    elif request.form['submit'] == 'Eliminar':
        return redirect(url_for('eliminar', messages=mensaje))
    elif request.form['submit'] == 'Recuperar':
        return redirect(url_for('recuperar', messages=mensaje))
    elif request.form['submit'] == 'Actualizar':
        return redirect(url_for('actualizar', messages=mensaje))
    return redirect(url_for('index'))

# Para eliminar la fila actual
@app.route('/eliminar', methods=['GET', 'POST'])
def eliminar():
    global df
    global aux
    if 'index' in session:
        index = int(session["index"])
        print(index)
        aux = df.copy()
        df = df.drop(index=[index])
        df = df.reset_index(drop=True)
        if df.shape[0] == 0:
            return 'Has eliminado el último registro, ya no queda nada por hacer'
        elif df.shape[0] == index:
            index -= 1
            session["index"] = index
        return redirect(url_for('index'))
    else:
        return redirect(url_for('index'))

@app.route('/recuperar', methods=['GET', 'POST'])
def recuperar():
    global df
    global aux
    if aux is not None:
        df = aux
        aux = None
    return redirect(url_for('index'))

# Cuando el usuario se marche persistimos los cambios
@socketio.on('disconnect')
@app.route('/salir', methods=['GET', 'POST'])
def salir():
    global df
    if 'index' in session and 'ident' in session:
        ident = session['ident']
        index = session["index"]
        guardar_cambios(df, index, "datos", ident)
    session.clear()
    
    return("¡Exito! Ya puede cerrar esta pestaña.")
    
# Leemos quién va a realizar la validación
@app.route('/submit', methods=['POST'])
def submit():
    session['ident'] = request.form['id']
    return redirect(url_for('index'))
    
# run the application 
if __name__ == "__main__": 
    socketio.run(app, debug=True, port=7860, use_reloader=False, allow_unsafe_werkzeug=True)

Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:7860
Press CTRL+C to quit
127.0.0.1 - - [28/May/2024 02:28:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2024 02:28:14] "GET /static/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2024 02:28:14] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/May/2024 02:29:21] "POST /generica HTTP/1.1" 302 -
127.0.0.1 - - [28/May/2024 02:29:21] "GET /recuperar?messages={"Pregunta":+"",+"Respuesta":+""} HTTP/1.1" 500 -
Traceback (most recent call last):
  File "/home/paascorb/.local/lib/python3.11/site-packages/flask/app.py", line 1478, in __call__
    return self.wsgi_app(environ, start_response)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/paascorb/.local/lib/python3.11/site-packages/flask_socketio/__init__.py", line 43, in __call__
    return super(_SocketIOMiddleware, self).__call__(environ,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/paascorb/.local/lib/python3.11/site-packages/engineio/middleware.py", line 

In [19]:
df.loc[0,['Pregunta']] = 'Hola'
df

,Pregunta,Respuesta
0,Hola,Pablo
1,Adios,Luis
2,Hasta luego,Pedro
